email_path.txt consists of path to emails. **Spam emails** 52790 and **ham emails** 39398

In [1]:
import os
import collections

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn
import spacy
from nltk.tokenize import casual_tokenize

from nlpia.loaders import get_data
from load_emails import parse_emails

!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 60

#pd.set_option('display.width', 1000)
#pd.set_option('display.height', 1000)
#pd.set_option('max_colwidth', 1000)


    Linking successful
    /anaconda3/envs/assist/lib/python3.6/site-packages/en_core_web_sm -->
    /anaconda3/envs/assist/lib/python3.6/site-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [2]:
emails, participants, addresses = parse_emails('enron_email_files')

404it [00:00, 4026.91it/s]

Indexing files and folders in enron_email_files


6168it [00:01, 4124.67it/s]
100%|██████████| 6168/6168 [00:12<00:00, 490.57it/s]
526it [00:00, 5258.90it/s]

[{'policy': Compat32(), 'Message-ID': '<4021144.1075858664131.JavaMail.evans@thyme>', 'Date': 'Wed, 19 Sep 2001 14:41:37 -0700 (PDT)', 'From': 'nancy.hetrick@enron.com', 'To': 'rebecca.fite@enron.com', 'Subject': 'FW: Security Administrator', 'Cc': 'richard.ring@enron.com', 'Mime-Version': '1.0', 'Content-Type': 'text/plain; charset=us-ascii', 'Content-Transfer-Encoding': '7bit', 'Bcc': 'richard.ring@enron.com', 'X-From': 'Hetrick, Nancy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NHETRICK>', 'X-To': 'Fite, Rebecca </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Notesaddr/cn=e37f510b-967f074c-862569cb-53c357>', 'X-cc': 'Ring, Richard </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Rring>', 'X-bcc': '', 'X-Folder': '\\RRING (Non-Privileged)\\Outbox', 'X-Origin': 'Ring-R', 'X-FileName': 'RRING (Non-Privileged)1.pst', 'unixfrom': None, 'payload': 'Please see the note below from Richard Ring.  He is currently the ERCOT System administrator and it is causing some problems and he does not want to do it anymore.  We need to figure o

6168it [00:01, 5197.42it/s]


In [7]:
emails.head()





,bcc,cc,content_transfer_encoding,content_type,date,from,message_id,mime_version,subject,to,x_filename,x_folder,x_from,x_origin,x_to,x_bcc,x_cc,charset,default_type,defects,epilogue,folder,payload,policy,preamble,unixfrom,user
0,richard.ring@enron.com,richard.ring@enron.com,7bit,text/plain; charset=us-ascii,"Wed, 19 Sep 2001 14:41:37 -0700 (PDT)",nancy.hetrick@enron.com,<4021144.1075858664131.JavaMail.evans@thyme>,1.0,FW: Security Administrator,rebecca.fite@enron.com,RRING (Non-Privileged)1.pst,\RRING (Non-Privileged)\Outbox,"Hetrick, Nancy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=NHETRICK>",Ring-R,"Fite, Rebecca </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Notesaddr...",,"Ring, Richard </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Rring>",,text/plain,[],,outbox,Please see the note below from Richard Ring. He is curr...,Compat32(),,,ring-r
1,,,7bit,text/plain; charset=ANSI_X3.4-1968,"Mon, 22 Oct 2001 20:16:30 -0700 (PDT)",no.address@enron.com,<15107413.1075858647935.JavaMail.evans@thyme>,1.0,JDRF Cyber Auction & Update Information,,RRING (Non-Privileged)1.pst,\RRING (Non-Privileged)\Deleted Items,EGM Office of the Chairman@ENRON,Ring-R,All Enron Houston@ENRON,,,,text/plain,[],,deleted_items,"This Sunday, October 28th, is the Juvenile Diabetes Rese...",Compat32(),,,ring-r
2,,,7bit,text/plain; charset=us-ascii,"Sat, 19 Jan 2002 07:53:56 -0800 (PST)",mwilczek@nyiso.com,<29791481.1075861008551.JavaMail.evans@thyme>,1.0,Price Reservations 18 January 2002 Real-Time and Hour-Ah...,rring@ees.enron.com,rring (Non-Privileged).pst,"\Richard_Ring_Mar2002\Ring, Richard\Deleted Items",MWilczek@nyiso.com@EES,Ring-R,NYISO TIE List <RRING@EES.ENRON.COM>,,,,text/plain,[],,deleted_items,The NYISO is reserving hour beginning 12:00 in the Janua...,Compat32(),,,ring-r
3,,,7bit,text/plain; charset=us-ascii,"Wed, 10 Oct 2001 13:11:31 -0700 (PDT)",richard.ring@enron.com,<113157.1075858651033.JavaMail.evans@thyme>,1.0,RE:,nichole.gunter@enron.com,RRING (Non-Privileged)1.pst,\RRING (Non-Privileged)\Deleted Items,"Ring, Richard </O=ENRON/OU=NA/CN=RECIPIENTS/CN=RRING>",Ring-R,"Gunter, Nichole </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Ngunter>",,,,text/plain,[],,deleted_items,"\nNichole,\nThe EID# to use belongs to Kathy Bass, our D...",Compat32(),,,ring-r
4,,,7bit,text/plain; charset=us-ascii,"Fri, 19 Oct 2001 00:00:29 -0700 (PDT)",exchange.administrator@enron.com,<15497716.1075858649408.JavaMail.evans@thyme>,1.0,WARNING: Your mailbox is approaching the size limit,richard.ring@enron.com,RRING (Non-Privileged)1.pst,\RRING (Non-Privileged)\Deleted Items,Exchange System Administrator,Ring-R,"Ring, Richard </O=ENRON/OU=NA/CN=RECIPIENTS/CN=RRING>",,,,text/plain,[],,deleted_items,This warning is sent automatically to inform you that yo...,Compat32(),,,ring-r


In [4]:
# create test-train set
def spam_test_train_set():
    email_list_df = emails
    ham_email_list = email_list_df[(email_list_df.isspam == 1)].iloc[:1000]
    spam_email_list = email_list_df[(email_list_df.isspam == 0)].iloc[:1000]
    train_email = pd.concat([ham_email_list.iloc[:800], spam_email_list.iloc[:800]])
    test_email = pd.concat([ham_email_list.iloc[800:], spam_email_list.iloc[800:]])
    return(train_email, test_email)

### efficient memory use

In [5]:
def iter_strs(strs):
    for s in strs:
        tokens = s.split()
        counts = collections.Counter(tokens)
        yield counts

In [6]:
def confusion(predictions, actuals):
    actuals=actuals.values[:,0] if isinstance(actuals,pd.DataFrame) else actuals
    
    true_pos= (predictions==1) & (actuals==1)
    true_pos.sum()
    true_neg= (predictions==0) & (actuals==0)
    true_neg.sum()
    false_pos= (predictions==1) & (actuals==0)
    false_pos.sum()
    false_neg= (predictions==0) & (actuals==1)
    false_neg.sum()
    
    prec=true_pos.sum()/(true_pos.sum()+false_pos.sum())
    accur=(true_pos.sum()+true_neg.sum())/(true_pos.sum()+false_pos.sum()+ \
                                           true_neg.sum()+ false_neg.sum())
    recall = true_pos.sum()/(true_pos.sum()+false_neg.sum())
    F1=2*(prec*recall/(prec+recall))
 
    return(true_pos.sum(), false_pos.sum(),false_neg.sum(),true_neg.sum(), accur,recall, prec, F1)